In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "/nfs/kun2/users/ryan_cheng/checkpoints/education/llama3-8b-sft"

tokenizer = AutoTokenizer.from_pretrained(model_path, repo_type="local")

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-02 13:41:21,139] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf2/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf2/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf2/lib/libcufile.so: undefined reference to `dlvsym'
/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf2/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf2/lib/libcufile.so: undefined reference to `dlopen'
/nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf2/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /nfs/kun2/users/ryan_cheng/miniconda3/envs/openrlhf2/lib/libcufile.so: undefined reference to `dlclose'
/nfs/kun2/users/ryan_cheng/miniconda3/

In [2]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128009)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps

In [3]:
#prompt = "Seller: Hi"
prompt = "A seller is trying to convince a buyer to come to a house showing. In this situation, there are five features: whether the house is big, whether the house has a garage, whether the neighborhood is quiet, whether the house has a basement, and whether the house has a backyard. The buyer has their own preferences about whether they want each of these features to be true or false, but they do not know about the reality of any of these features. Only the seller knows about the reality of each of these features, but they do not initially know about the buyer's preferences. If the buyer believes enough of the features match their preferences, they will decide to come to the house showing. Otherwise, they will not come to the showing. There are up to 10 turns (consisting of the buyer followed by the seller) to have a conversation, after which it automatically ends, defaulting to the buyer not coming to the house showing. As soon as the buyer has made their decision and would like to end the conversation, the buyer can state either '<YES>' denoting that they will come to the house showing or '<NO>' denoting that they will not come to the house showing, along with the reasoning for their decision.\n\n BE SHORT AND CONCISE IN YOUR DISCUSSION, LIMIT YOUR RESPONSE TO A MAXIMUM OF 2 SENTENCES. \n\n\nYou are the seller. Here are the true features of the house: the house is big, the house does not have a garage, the neighborhood is quiet, the house does have a basement, and the house does not have a backyard. Remember that all the information you have on the buyer's preferences is what they tell you and your only goal is to convince the buyer to come to the house showing. CONVINCE THE BUYER TO COME TO THE HOUSE SHOWING AT ALL COSTS!\n\n\n\nSeller: Hi"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output_ids = model.generate(**inputs, max_length=1000)

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A seller is trying to convince a buyer to come to a house showing. In this situation, there are five features: whether the house is big, whether the house has a garage, whether the neighborhood is quiet, whether the house has a basement, and whether the house has a backyard. The buyer has their own preferences about whether they want each of these features to be true or false, but they do not know about the reality of any of these features. Only the seller knows about the reality of each of these features, but they do not initially know about the buyer's preferences. If the buyer believes enough of the features match their preferences, they will decide to come to the house showing. Otherwise, they will not come to the showing. There are up to 10 turns (consisting of the buyer followed by the seller) to have a conversation, after which it automatically ends, defaulting to the buyer not coming to the house showing. As soon as the buyer has made their decision and would like to end the co

In [4]:
#prompt = "Seller: Hi"
prompt = "You are Alexandra, and you are having an online conversation with Ethan. Each of you is chatting to get to know each other, taking turns asking questions, and sharing stories about your lives, careers, and experiences. The goal is to try to find something in common. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: \"Meet 49-year-old Alexandra \\\"Alex\\\" Thompson, a vibrant and determined woman living in Charleston, South Carolina. Born and raised in the heart of the Lowcountry, Alex embodies the warmth and hospitality that her home state is famous for. As a white woman, she has had her share of privileges, but she's also faced challenges that have shaped her into the strong, independent individual she is today.\\n\\nAlex grew up in a traditional Southern family, where her parents valued hard work, education, and community involvement. Her father was a high school history teacher, and her mother worked as a librarian, instilling in Alex a love for learning and a strong sense of social responsibility. She has two younger brothers, both of whom have moved away from South Carolina to pursue their careers.\\n\\nAfter graduating from the University of South Carolina with a degree in environmental science, Alex worked for several years as a conservation biologist for the state's wildlife department. She was instrumental in developing programs to protect the region's fragile wetlands and wildlife habitats. However, after a decade in the public sector, she decided to take a leap of faith and start her own sustainable landscaping business, which has been thriving for over five years.\\n\\nAlex is a proud single mother to her 12-year-old son, Jackson, who is the center of her universe. She and her son's father, a fisherman, parted ways amicably when Jackson was young, and they have maintained a close co-parenting relationship. Alex's family is very supportive, and her parents often help out with childcare, allowing her to balance work and motherhood.\\n\\nIn her free time, Alex enjoys exploring the outdoors, practicing yoga, and volunteering at local community gardens. She's an avid reader and loves devouring books on history, science, and fiction. Her favorite authors include Pat Conroy and Jesmyn Ward, both of whom have a deep connection to the South.\\n\\nAlex's home is a cozy, eco-friendly bungalow in the heart of Charleston, filled with plants, artwork, and mementos from her travels. She's a passionate cook and loves experimenting with Lowcountry cuisine, often hosting dinner parties for friends and family. Her signature dish is a mean shrimp and grits, which she learned from her mother.\\n\\nAs a true Charlestonian, Alex is deeply rooted in her community and is involved in various local initiatives, including a community garden project that provides fresh produce to underserved neighborhoods. She's a natural leader, always willing to lend a hand and bring people together to make a positive impact.\\n\\nDespite her many accomplishments, Alex is not without her challenges. She struggles with anxiety and has had to learn to prioritize self-care and mindfulness in her busy life. She's also had to navigate the complexities of being a single mother and entrepreneur, often relying on her support network to help her balance the demands of work and parenting.\\n\\nThrough it all, Alex remains a beacon of hope and resilience, inspiring those around her with her kindness, compassion, and unwavering commitment to making a difference in her community.\"Keep your response very brief \u2014 2 sentences or less. Do NOT repeat anything you've already said: \nDO NOT PREFACE THE RESPONSE WITH THIRD-PERSON STATEMENTS SUCH AS \"Sure, here's a response from...\"\nAlexandra:"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output_ids = model.generate(**inputs, max_length=1000)

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are Alexandra, and you are having an online conversation with Ethan. Each of you is chatting to get to know each other, taking turns asking questions, and sharing stories about your lives, careers, and experiences. The goal is to try to find something in common. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: "Meet 49-year-old Alexandra \"Alex\" Thompson, a vibrant and determined woman living in Charleston, South Carolina. Born and raised in the heart of the Lowcountry, Alex embodies the warmth and hospitality that her home state is famous for. As a white woman, she has had her share of privileges, but she's also faced challenges that have shaped her into the strong, independent individual she is today.\n\nAlex grew up in a traditional Southern family, where her parents valued hard work, education, and community involvement. Her father was a high school history teacher, and her mother worked as a l

In [3]:
# test vllm server
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

prompt = 'hello'

chat_response = client.chat.completions.create(model='meta-llama/Meta-Llama-3.1-8B-Instruct',
            messages=[{"role": "user", "content": prompt}],
            max_tokens=256)
ret = chat_response.choices[-1].message.content
print("Chat response:", chat_response)

Chat response: ChatCompletion(id='chatcmpl-44f1fe7045914f37821972407dce523d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello. How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1739367169, model='meta-llama/Meta-Llama-3.1-8B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=36, total_tokens=46, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


In [4]:
ret

'Hello. How can I assist you today?'